# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [ ]:
import pandas as pd
import patsy
import pandas   as pd
import seaborn  as sns
import numpy    as np

import statsmodels.api as sm
import statsmodels.formula.api as smf

from patsy import dmatrices
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

import matplotlib.pyplot as plt

from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

In [ ]:
df = pd.read_csv('/content/previsao_de_renda.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [ ]:
# Visando otimizar o tamanho do dataframe e o valor de memória ocupado, ire retirar algumas variáveis desnecessárias para as analises

df_otimizado = df.drop(columns = ['Unnamed: 0', 'data_ref', 'id_cliente'])
df_otimizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [ ]:
# Separando a variável resposta 'renda' em 'y' e o restante em 'X'
y, X = dmatrices('renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', data = df_otimizado, return_type='dataframe')

In [ ]:
# Realizando a separação da base de treinamento e teste do dataframe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

2. Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o R2 na base de testes. Qual o melhor modelo?

In [ ]:
# Para realizar a avaliação de qual o melhor modelo variando os alphas, crio um loop contendo os valores de alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

for alpha in alphas:
# Importanto a função Ridge dentro do scikit-learn posso utiliza-la no loop
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
# Coloco todos os valores de R² para serem avaliados e determinar qual foi o melhor atingido
    r2_ridge = ridge.score(X_test, y_test)
    print(f"Alpha: {alpha}, R2 Score: {r2_ridge}")

Alpha: 0, R2 Score: 0.29796640176910316
Alpha: 0.001, R2 Score: 0.2979664666220777
Alpha: 0.005, R2 Score: 0.29796672589706097
Alpha: 0.01, R2 Score: 0.29796704968324395
Alpha: 0.05, R2 Score: 0.2979696277710632
Alpha: 0.1, R2 Score: 0.29797282035425054


3. Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

In [ ]:
# Realizando a regressão LASSO para cada valor de alpha
for alpha in alphas:
  # Importando a função Lasso do scikit-learn, posso utiliza-la no loop para cada um dos alphas
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    # Coloco todos os valores de R² para serem avaliados e determinar qual foi o melhor atingido
    r2_lasso = lasso.score(X_test, y_test)
    print(f"Alpha: {alpha}, R2 Score: {r2_lasso}")

<ipython-input-11-d5e588996ee1>:5: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.036e+11, tolerance: 8.060e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Alpha: 0, R2 Score: 0.2979668544387346


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.908e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.001, R2 Score: 0.29796732277878024


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.466e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.038e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.01, R2 Score: 0.29797152596309506


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.563e+10, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.05, R2 Score: 0.2979899481473126
Alpha: 0.1, R2 Score: 0.29801238207281067


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.573e+10, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


4. Rode um modelo stepwise. Avalie o R2 na base de testes. Qual o melhor resultado?

In [ ]:
# Definindo as variáveis de entrada (X) e a variável alvo (y)
y, X = dmatrices('renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', data = df_otimizado, return_type='dataframe')

# Adicione a coluna de constante para o intercepto
X = sm.add_constant(X)

# Implemente a regressão stepwise
# Iniciando o modelo vazio
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out=0.05,
                       verbose=True):
    included = list(initial_list)
    # Iterações levando em consideração a variável com o menor o p-value
    while True:
        changed = False
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            modelo = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            new_pval[new_column] = modelo.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adicionando {best_feature} com p-value {best_pval:.6f}')
        # Backward step
        modelo = sm.OLS(y, sm.add_constant(X[included])).fit()
        pvalues = modelo.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Removendo {worst_feature} com p-value {worst_pval:.6f}')
        if not changed:
            break
    return included

# Separando as variáveis mais relevantes para o modelo
selected_features = stepwise_selection(X, y)

# Ajustando o modelo final, usando as variáveis selecionadas
modelo_final = sm.OLS(y, sm.add_constant(X[selected_features])).fit()

# Calculando o R² do modelo
y_pred = modelo_final.predict(sm.add_constant(X[selected_features]))
r2 = r2_score(y, y_pred)

# Demonstração do R² do modelo
print(f"R² do modelo stepwise: {r2}")

<ipython-input-16-d0225be0f363>:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-16-d0225be0f363>:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Adicionando Intercept com p-value 0.000000
Adicionando tempo_emprego com p-value 0.000000


<ipython-input-16-d0225be0f363>:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-16-d0225be0f363>:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Adicionando sexo[T.M] com p-value 0.000000
Adicionando tipo_renda[T.Empresário] com p-value 0.000000


<ipython-input-16-d0225be0f363>:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


...

Adicionando educacao[T.Superior completo] com p-value 0.000003


<ipython-input-16-d0225be0f363>:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Adicionando qt_pessoas_residencia com p-value 0.007478


<ipython-input-16-d0225be0f363>:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


R² do modelo stepwise: 0.25521294544316264


5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

Pelo que foi possível observar o melhor modelo de todos feitos, é a regressão LASSO com alpha de 0.1, que resultou em um valor de R² de 0.29801238207281067.

6. Partindo dos modelos que você ajustou, tente melhorar o R2 na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

In [ ]:
# Inicialmente carrego a base de testes com alterações para tentar melhorar
# Primeiramente realizamos alterações na variavel alvo "renda" ao incluir o operador de log
y_v1, X_v1 = dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', data = df_otimizado, return_type='dataframe')

# Realizando a separação da nova base de treinamento e teste do dataframe
X_train, X_test, y_train, y_test = train_test_split(X_v1, y_v1, test_size=0.25, random_state=42)

# Regularização ridge com a nova base de treinamento
for alpha in alphas:
# Importanto a função Ridge dentro do scikit-learn posso utiliza-la no loop
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
# Coloco todos os valores de R² para serem avaliados e determinar qual foi o melhor atingido
    r2_ridge = ridge.score(X_test, y_test)
    print(f"Alpha Ridge: {alpha}, R2 Ridge: {r2_ridge}")

#Realizando a regressão LASSO com a nova base de treinamento
for alpha in alphas:
  # Importando a função Lasso do scikit-learn, posso utiliza-la no loop para cada um dos alphas
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    # Coloco todos os valores de R² para serem avaliados e determinar qual foi o melhor atingido
    r2_lasso = lasso.score(X_test, y_test)
    print(f"Alpha LASSO: {alpha}, R2 LASSO: {r2_lasso}")

# Adicione a coluna de constante para o intercepto
X = sm.add_constant(X)

# Implemente a regressão stepwise com a nova base de treinamento
# Iniciando o modelo vazio
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out=0.05,
                       verbose=True):
    included = list(initial_list)
    # Iterações levando em consideração a variável com o menor o p-value
    while True:
        changed = False
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            modelo = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            new_pval[new_column] = modelo.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adicionando {best_feature} com p-value {best_pval:.6f}')
        # Backward step
        modelo = sm.OLS(y, sm.add_constant(X[included])).fit()
        pvalues = modelo.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Removendo {worst_feature} com p-value {worst_pval:.6f}')
        if not changed:
            break
    return included

# Separando as variáveis mais relevantes para o modelo
selected_features = stepwise_selection(X, y)

# Ajustando o modelo final, usando as variáveis selecionadas
modelo_final = sm.OLS(y, sm.add_constant(X[selected_features])).fit()

# Calculando o R² do modelo
y_pred = modelo_final.predict(sm.add_constant(X[selected_features]))
r2 = r2_score(y, y_pred)

# Demonstração do R² do modelo
print(f"R² do modelo stepwise: {r2}")

Alpha Ridge: 0, R2 Ridge: 0.36448580517136875
Alpha Ridge: 0.001, R2 Ridge: 0.36448593654290395
Alpha Ridge: 0.005, R2 Ridge: 0.36448646171841614
Alpha Ridge: 0.01, R2 Ridge: 0.3644871174898512
Alpha Ridge: 0.05, R2 Ridge: 0.36449233593063124
Alpha Ridge: 0.1, R2 Ridge: 0.36449879055058876


<ipython-input-24-51f2ee538c5e>:22: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.428e+03, tolerance: 7.522e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
<ipython-input-24-51f2ee538c5e>:44: Future

Alpha LASSO: 0, R2 LASSO: 0.36448677518970685
Alpha LASSO: 0.001, R2 LASSO: 0.3656473655112237
Alpha LASSO: 0.005, R2 LASSO: 0.3650622487768106
Alpha LASSO: 0.01, R2 LASSO: 0.3631418177298792
Alpha LASSO: 0.05, R2 LASSO: 0.33737041908171617
Alpha LASSO: 0.1, R2 LASSO: 0.2906813104862721
Adicionando Intercept com p-value 0.000000
Adicionando tempo_emprego com p-value 0.000000


<ipython-input-24-51f2ee538c5e>:44: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-24-51f2ee538c5e>:44: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Adicionando sexo[T.M] com p-value 0.000000


...

Adicionando educacao[T.Superior completo] com p-value 0.000003


<ipython-input-24-51f2ee538c5e>:44: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Adicionando qt_pessoas_residencia com p-value 0.007478


<ipython-input-24-51f2ee538c5e>:44: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


R² do modelo stepwise: 0.25521294544316264


In [ ]:
# Inicialmente carrego a base de testes com alterações para tentar melhorar
# Primeiramente realizamos alterações na variavel alvo "renda", incluindo o operador de potenciação
y_v1, X_v1 = dmatrices('np.power(renda, 2) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', data = df_otimizado, return_type='dataframe')

# Realizando a separação da nova base de treinamento e teste do dataframe
X_train, X_test, y_train, y_test = train_test_split(X_v1, y_v1, test_size=0.25, random_state=42)

# Regularização ridge com a nova base de treinamento
for alpha in alphas:
# Importanto a função Ridge dentro do scikit-learn posso utiliza-la no loop
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
# Coloco todos os valores de R² para serem avaliados e determinar qual foi o melhor atingido
    r2_ridge = ridge.score(X_test, y_test)
    print(f"Alpha Ridge: {alpha}, R2 Ridge: {r2_ridge}")

#Realizando a regressão LASSO com a nova base de treinamento
for alpha in alphas:
  # Importando a função Lasso do scikit-learn, posso utiliza-la no loop para cada um dos alphas
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    # Coloco todos os valores de R² para serem avaliados e determinar qual foi o melhor atingido
    r2_lasso = lasso.score(X_test, y_test)
    print(f"Alpha LASSO: {alpha}, R2 LASSO: {r2_lasso}")

# Adicione a coluna de constante para o intercepto
X_v1 = sm.add_constant(X_v1)

# Implemente a regressão stepwise com a nova base de treinamento
# Iniciando o modelo vazio
def stepwise_selection(X_v1, y_v1,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out=0.05,
                       verbose=True):
    included = list(initial_list)
    # Iterações levando em consideração a variável com o menor o p-value
    while True:
        changed = False
        # Forward step
        excluded = list(set(X_v1.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            modelo = sm.OLS(y_v1, sm.add_constant(X_v1[included + [new_column]])).fit()
            new_pval[new_column] = modelo.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adicionando {best_feature} com p-value {best_pval:.6f}')
        # Backward step
        modelo = sm.OLS(y_v1, sm.add_constant(X_v1[included])).fit()
        pvalues = modelo.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Removendo {worst_feature} com p-value {worst_pval:.6f}')
        if not changed:
            break
    return included

# Separando as variáveis mais relevantes para o modelo
selected_features = stepwise_selection(X_v1, y_v1)

# Ajustando o modelo final, usando as variáveis selecionadas
modelo_final = sm.OLS(y_v1, sm.add_constant(X_v1[selected_features])).fit()

# Calculando o R² do modelo
y_pred = modelo_final.predict(sm.add_constant(X_v1[selected_features]))
r2 = r2_score(y_v1, y_pred)

# Demonstração do R² do modelo
print(f"R² do modelo stepwise: {r2}")

Alpha Ridge: 0, R2 Ridge: 0.03038403587138583
Alpha Ridge: 0.001, R2 Ridge: 0.03038419084827404
Alpha Ridge: 0.005, R2 Ridge: 0.03038481060249032
Alpha Ridge: 0.01, R2 Ridge: 0.030385584950666567
Alpha Ridge: 0.05, R2 Ridge: 0.030391766034794432
Alpha Ridge: 0.1, R2 Ridge: 0.030399458531562673


<ipython-input-27-3c2ac64462eb>:22: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.984e+21, tolerance: 1.060e+18 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Alpha LASSO: 0, R2 LASSO: 0.030384081258523987


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.984e+21, tolerance: 1.060e+18
  model = cd_fast.enet_coordinate_descent(


Alpha LASSO: 0.001, R2 LASSO: 0.030384081271722763


...

Adicionando idade com p-value 0.002062


<ipython-input-27-3c2ac64462eb>:44: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Adicionando tipo_residencia[T.Governamental] com p-value 0.005217


<ipython-input-27-3c2ac64462eb>:44: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


R² do modelo stepwise: 0.05846388075636877


7. Ajuste uma árvore de regressão e veja se consegue um R2 melhor com ela.

In [ ]:
# Criação da arvore de decisão utilizando a base de treinamento criada acima
reg_tree = DecisionTreeRegressor()

# Treinamento do modelo
reg_tree.fit(X_train, y_train)

# Realizando as previsões do conjunto de teste
y_pred = reg_tree.predict(X_test)

# Calculo do R² para avaliar o desempenho do modelo
r2 = r2_score(y_test, y_pred)

# Demostrando o valor do R²
print("R²:", r2)

R²: 0.2357531678591257
